In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.plotting.backend = "plotly"


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/banking-dataset-marketing-targets/train.csv", sep=";")
test_df = pd.read_csv("/kaggle/input/banking-dataset-marketing-targets/test.csv", sep=";")
rain_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/banking-dataset-marketing-targets/train.csv'

In [ ]:
train_df.info()

NameError: name 'train_df' is not defined

In [ ]:
test_df.shape

In [ ]:
df = train_df.copy()

In [ ]:
df['y'].value_counts()

In [ ]:
test_size = test_df.shape[0]
train_size = train_df.shape[0]

In [ ]:
yes_df = train_df[train_df['y'] == 'yes']
no_df = train_df[train_df['y'] == 'no']

df = pd.concat([yes_df, no_df.sample(int(no_df.shape[0] * .5))])
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
# df22 = df.groupby('y').sample(5289).reset_index(drop=True)
# df22 = df22.sample(frac=1).reset_index(drop=True)
# df22.head()
# df = df22

In [ ]:
# df22['y'].value_counts()

In [ ]:
df['y'].value_counts()

In [ ]:
num_cols = df.select_dtypes("number").columns
num_cols

In [ ]:
str_cols = df.select_dtypes("O").columns
str_cols

In [ ]:
df[num_cols].describe().T.iloc[:, 1:]

In [ ]:
str_des = df[str_cols].describe().T.iloc[:, 1:]
str_des

In [ ]:
binary_cols = str_des[(str_des['unique'] == 2)].index
for col in binary_cols:
    print(f"{col}:\n{df[col].unique()}\n")

In [ ]:
df.hist(figsize=(10, 10), x='age', title='Age distribution')

In [ ]:
df.groupby('age')['y'].value_counts(
    normalize=True).unstack().plot(kind='bar', title='The Y value distribution in each age category')


In [ ]:
df['age_cat'] = pd.cut(df.age,
                       bins=[0, 30, 60, np.inf],
                       labels=[1, 2, 3]
                       ).astype(np.uint8)

In [ ]:
df_grouped = df.groupby('age_cat')['y'].value_counts(
    normalize=True).unstack().T
df_grouped

In [ ]:
# Create a subplot layout
fig = make_subplots(rows=1,
                    cols=len(df_grouped.columns),
                    specs=[
                        [
                            {'type': 'pie'}
                        ] * len(df_grouped.columns)
                    ],
                    subplot_titles=df_grouped.columns.astype(str),

                    )

# Iterate through each age category and add a pie chart to the subplot
for i, age_cat in enumerate(df_grouped.columns):
    data = df_grouped[age_cat]
    fig.add_trace(
        go.Pie(labels=data.index, values=data.values,
               name=f'age_cat: {age_cat}', title=f'Age Category'),
        row=1,
        col=i+1
    )

# Update layout
fig.update_layout(
    title_text='Y value distribution in each age category',
)

In [ ]:
df['age_cat'] = pd.cut(df.age,
                       bins=[0, 30, 60, np.inf],
                       labels=[1, 2, 3]
                       ).astype(np.uint8)

In [ ]:
df_grouped = df.groupby('age_cat')['y'].value_counts(
    normalize=True).unstack().T
df_grouped

In [ ]:
# Create a subplot layout
fig = make_subplots(rows=1,
                    cols=len(df_grouped.columns),
                    specs=[
                        [
                            {'type': 'pie'}
                        ] * len(df_grouped.columns)
                    ],
                    subplot_titles=df_grouped.columns.astype(str),

                    )

# Iterate through each age category and add a pie chart to the subplot
for i, age_cat in enumerate(df_grouped.columns):
    data = df_grouped[age_cat]
    fig.add_trace(
        go.Pie(labels=data.index, values=data.values,
               name=f'age_cat: {age_cat}', title=f'Age Category'),
        row=1,
        col=i+1
    )

# Update layout
fig.update_layout(
    title_text='Y value distribution in each age category',
)

In [ ]:
df_grouped = df.groupby('age_cat', as_index=False)['balance'].describe()
df_grouped

In [ ]:
fig = px.bar(df_grouped,
             x='age_cat',
             y='mean',
             color='count',
             title='Mean balance in each age category and it\'s count',)

fig.update_layout(
    xaxis_title="Age Category",
    yaxis_title="Mean Balance",
)

In [ ]:
num_cols

In [ ]:
str_cols

In [ ]:
df_grouped = df.groupby('job')['balance'].describe(
).iloc[:, :2].sort_values(by=['mean', 'count'], ascending=False)
df_grouped

In [ ]:
job_age_ = df.groupby('job')['age'].mean().rename(
    'mean_age').sort_values(ascending=False)
job_age_

In [ ]:
from typing import Literal


def label_age_group(age) -> None | Literal[1] | Literal[2] | Literal[3]:

    if age < 30:
        return 1
    elif age <= 60:
        return 2
    elif age <= 90 or age > 90:
        return 3

In [ ]:
# Create a subplot layout

from numpy import around


df_grouped['age_group'] = job_age_.map(label_age_group).rename('age_group')


fig = make_subplots(rows=2,
                    cols=2,
                    specs=[

                        [
                            {'type': 'bar'},
                            {'type': 'pie'}
                        ],
                        [
                            {'type': 'bar'},
                            {'type': 'bar'}
                        ]
                    ],
                    subplot_titles=[
                        'Mean Balance for each job category',
                        'Count Percentage',
                        'Mean Age Group for each job category',
                        'Percentage Yes for each jobs'
                    ],
                    )


fig.add_trace(
    go.Bar(x=df_grouped.index,
           y=df_grouped['mean'],
           name='Mean Balance'),
    row=1,
    col=1,
)

fig.add_trace(
    go.Pie(labels=df_grouped.index,
           values=df_grouped['count']),
    row=1,
    col=2
)

fig.add_trace(
    go.Bar(x=df_grouped.index,
           y=df_grouped['age_group'],
           name='age_group', text=df_grouped['age_group']),
    row=2,
    col=1,
)

yes_job_percentage = (df[df['y'] == 'yes']['job']
                      .value_counts() /
                      df['job'].value_counts()
                      ).sort_values(ascending=False)


fig.add_trace(
    go.Bar(x=yes_job_percentage.index, y=yes_job_percentage, name='Yes Percentage',
           text=round(yes_job_percentage*100).astype(str) + ' %', textposition='auto'),
    row=2,
    col=2,
)

fig.update_layout(height=800, width=1000)

In [ ]:
num_cols

In [ ]:
str_cols

In [ ]:
yes_marital_percentage = df[df['y'] == 'yes']['marital'].value_counts(
)/df['marital'].value_counts()
yes_marital_percentage

In [ ]:
grouped_df = df.groupby(by=['marital', 'y'])['balance'].mean(
).sort_values(ascending=False).reset_index()
grouped_df

In [ ]:
# Creating bar traces for each 'y' category
yes_trace = go.Bar(
    x=grouped_df[grouped_df['y'] == 'yes']['marital'],
    y=grouped_df[grouped_df['y'] == 'yes']['balance'],
    name='yes',
    marker=dict(color='lightblue')
)

no_trace = go.Bar(
    x=grouped_df[grouped_df['y'] == 'no']['marital'],
    y=grouped_df[grouped_df['y'] == 'no']['balance'],
    name='no',
    marker=dict(color='orange')
)

In [ ]:
yes_marital_percentage = df[df['y'] == 'yes']['marital'].value_counts(
) / df['marital'].value_counts()
yes_marital_percentage

In [ ]:
yes_marital_percentage_trace = go.Pie(
    labels=yes_marital_percentage.index,
    values=yes_marital_percentage.values,
)

In [ ]:
fig = make_subplots(
    rows=1,
    cols=2,
    specs=[
        [
            {'type': 'xy'},
            {'type': 'pie'}
        ]
    ],
    subplot_titles=[
        'Mean Balance for each Marital Status Grouped By Yes/No',
        'Percentage Yes for each marital status',
    ]
)

fig.add_trace(
    yes_trace,
    row=1,
    col=1
)
fig.add_trace(
    no_trace,
    row=1,
    col=1
)

fig.add_trace(
    yes_marital_percentage_trace,
    row=1,
    col=2
)

In [ ]:
num_cols

In [ ]:
str_cols

In [ ]:
yes_marital_percentage = df[df['y'] == 'yes']['marital'].value_counts(
)/df['marital'].value_counts()
yes_marital_percentage

In [ ]:
grouped_df = df.groupby(by=['marital', 'y'])['balance'].mean(
).sort_values(ascending=False).reset_index()
grouped_df

In [ ]:
# Creating bar traces for each 'y' category
yes_trace = go.Bar(
    x=grouped_df[grouped_df['y'] == 'yes']['marital'],
    y=grouped_df[grouped_df['y'] == 'yes']['balance'],
    name='yes',
    marker=dict(color='lightblue')
)

no_trace = go.Bar(
    x=grouped_df[grouped_df['y'] == 'no']['marital'],
    y=grouped_df[grouped_df['y'] == 'no']['balance'],
    name='no',
    marker=dict(color='orange')
)

In [ ]:
yes_marital_percentage = df[df['y'] == 'yes']['marital'].value_counts(
) / df['marital'].value_counts()
yes_marital_percentage

In [ ]:
yes_marital_percentage_trace = go.Pie(
    labels=yes_marital_percentage.index,
    values=yes_marital_percentage.values,
)

In [ ]:
fig = make_subplots(
    rows=1,
    cols=2,
    specs=[
        [
            {'type': 'xy'},
            {'type': 'pie'}
        ]
    ],
    subplot_titles=[
        'Mean Balance for each Marital Status Grouped By Yes/No',
        'Percentage Yes for each marital status',
    ]
)

fig.add_trace(
    yes_trace,
    row=1,
    col=1
)
fig.add_trace(
    no_trace,
    row=1,
    col=1
)

fig.add_trace(
    yes_marital_percentage_trace,
    row=1,
    col=2
)

In [ ]:
num_cols

In [ ]:
str_cols

In [ ]:
options_ = ['yes', 'no']

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [
            {'type': 'pie'},
            {'type': 'pie'}
        ],
        [
            {'type': 'pie'},
            {'type': 'pie'}
        ]
    ],
    subplot_titles=[
        'loan: {}, housing: {}'.format(loan_, housing_)
        for loan_ in options_
        for housing_ in options_
    ]
)


for i, housing_ in enumerate(options_):
    for e, loan_ in enumerate(options_):
        temp = df[(df['loan'] == loan_) & (df['housing'] == housing_)
                  ]['y'].value_counts(normalize=True)

        fig.add_trace(
            go.Pie(
                labels=temp.index,
                values=temp.values,
                name='loan: {}, housing: {}'.format(loan_, housing_),
            ),
            row=i+1, col=e+1)

fig.update_layout(
    height=700,
    width=1300,
    title_text='Loan vs. Housing vs. Yes/No',
    legend_title_text='y'

)

In [ ]:
t = (df.groupby(by=['loan', 'housing'])[
    'default'].value_counts(normalize=True).sort_values(ascending=False)*100).reset_index()
t

In [ ]:
t_g = t[t['default'] == 'yes'].drop('default', axis=1)
t_g['names'] = f'loan= ' + t_g['loan'] + ', housing= ' + t_g['housing']
t_g

In [ ]:
dff2 = pd.get_dummies(dff2, columns=['job', 'marital', 'contact', 'poutcome'])

In [ ]:
rc = RandomForestClassifier()

rc.fit(x_train, y_train)

rc.score(x_test, y_test)

In [ ]:
print(classification_report(y_test, rc.predict(x_test)))